In [ ]:
import pandas as pd
import numpy as np
import config as cf
import matplotlib.pyplot as plt
import talib

In [ ]:
from datetime import datetime

### Loading data

In [ ]:
%store -r tickers

In [ ]:
tickers

In [ ]:
dfs = dict()
print("...loading...")
for ticker in tickers:
    symbol = ticker["symbol"]
    if str(symbol).endswith("USDT"):
        print(symbol, end=" ")
        filename = f"{cf.AGGREGATED_DATA_PATH}futures/{symbol}-4h.h5"
        df = pd.read_hdf(filename)
        dfs[symbol] = df

In [ ]:
len(dfs)

##### Pick AAAUSDT symbol which has history data length greater than 600 days

In [ ]:
len_list = []
for symbol in dfs.keys():
    len_list.append(len(dfs[symbol]))
length_df = pd.DataFrame({"symbol": dfs.keys(), "length": len_list})

In [ ]:
length_df

In [ ]:
len(length_df[length_df["length"]>600])

##### The Strategy is we arbitrage 5 pairs, with long top 5 that has the highest accumulated return in the last period, and short 5 pairs that has the lowest cummulated return in the last period.

##### Prepare data

In [ ]:
TRADE_TIMEFRAME_IN_MS = 4 * 60 * 60 * 1000

def preprocessing(trade_duration,lookback_period,holding_period,rsi_period,macd_fast,macd_slow,macd_signal,length_df, dfs):
    selected_symbols = length_df[length_df["length"]>trade_duration+lookback_period+holding_period]["symbol"].values.tolist()
    for symbol in selected_symbols:
        df = dfs[symbol]
        df[f"{symbol}_log_return"] = np.log(df["Close"].pct_change()+1)
        df[f"{symbol}_rsi"] = talib.RSI(df["Close"], timeperiod=rsi_period)
        _,_,df[f"{symbol}_macd"] = talib.MACD(df["Close"],fastperiod=macd_fast,slowperiod=macd_slow,signalperiod=macd_signal) 
        df[f"{symbol}_future_return"] = df[f"{symbol}_log_return"].shift(-holding_period).rolling(holding_period).sum()
        
    merged_df = pd.concat([dfs[symbol][[f"{symbol}_log_return",\
        f"{symbol}_future_return", f"{symbol}_rsi",f"{symbol}_macd"]] for symbol in selected_symbols],axis=1)
    
    merged_df.sort_index(ascending=True,inplace=True)
    merged_df.fillna(method="ffill", inplace = True)
    merged_df.dropna(inplace=True)

    return merged_df, selected_symbols

def calculate_discounted_sum(df,symbol,index,alpha,period):
    price_list = df.loc[index-period*TRADE_TIMEFRAME_IN_MS+1:index][f"{symbol}_log_return"].to_list()
    alpha_list = [alpha ** i for i in range(period-1,-1,-1)]
    return np.dot(price_list,alpha_list)

In [303]:
def run_strategy(
        trade_duration:int,
        length_df,
        dfs,
        lookback_period:int = 7 * 6,
        holding_period:int = 3 * 6,
        avoid_top:int = 3,
        n_pairs:int = 5,
        rsi_min = 25,
        rsi_max = 75,
        rsi_period = 14,
        macd_fast = 25,
        macd_slow = 12,
        macd_signal = 9,
        alpha = 0.97,
        balance:int =100_000,
        trading_fee:float=0.0004,
       ):
   
    merged_df, selected_symbols = preprocessing(
        trade_duration=trade_duration,
        lookback_period=lookback_period,
        holding_period=holding_period,
        length_df=length_df,
        rsi_period=rsi_period,
        macd_fast=macd_fast,
        macd_slow=macd_slow,
        macd_signal=macd_signal,
        dfs = dfs)
    
    trade_start = merged_df.index.min() + lookback_period * TRADE_TIMEFRAME_IN_MS
    trade_end = merged_df.index.max()
    
    #Check if any missing open-time index
    for i in range (trade_start,trade_end,TRADE_TIMEFRAME_IN_MS):
        if not i in merged_df.index:
            print(f"miss {i}")
    
    trade_df = pd.DataFrame({"open":[trade_start],"symbol":[""],"type":[""],"profit":[0],"cost":[0],"balance":[balance]})

    balance = balance
    profit_before_cost = 0
    
    for open_time in range(trade_start,trade_end+1, holding_period * TRADE_TIMEFRAME_IN_MS):
        
        date_time = datetime.utcfromtimestamp(open_time/1000).strftime('%Y-%m-%d %H:%M:%S')
        print(date_time)
        row = merged_df.loc[open_time,:].copy()

        long_candidates = []
        short_candidates = []
           
        for symbol in selected_symbols:
            if ((row[f"{symbol}_rsi"] < rsi_max) & (row[f"{symbol}_macd"] > 0)):
                long_candidates.append(symbol)
            
            if ((row[f"{symbol}_rsi"] > rsi_min) & (row[f"{symbol}_macd"] < 0)):
                short_candidates.append(symbol)
        
        long_history_profits = dict()
        
        for symbol in long_candidates:
            profit = calculate_discounted_sum(merged_df,symbol,open_time,alpha,lookback_period)
            long_history_profits[profit] = symbol
        sorted_long_symbol_list = sorted(long_history_profits,reverse=False)
        long_symbols = [long_history_profits[sorted_long_symbol_list[i]] \
            for i in range(min(len(long_candidates),n_pairs))]
        print(f"LONG {long_symbols}")
        
        short_history_profits = dict()
        for symbol in short_candidates:
            profit = calculate_discounted_sum(merged_df,symbol,open_time,alpha,lookback_period)
            short_history_profits[profit] = symbol
        sorted_short_symbol_list = sorted(short_history_profits,reverse=True)
        short_symbols = [short_history_profits[sorted_short_symbol_list[i]] \
            for i in range(min(len(short_candidates),n_pairs))]
        print(f"SHORT {short_symbols}")
        
        trade_symbols = []
        trade_types = []
        trade_costs = []
        trade_profits = []
        balances = []

        trade_num = len(long_symbols)+len(short_symbols)
        invest_amount = balance/trade_num if trade_num > 0 else 0
         
        for symbol in long_symbols:
            cumsum_log_return = merged_df.loc[open_time,:][f"{symbol}_future_return"]
            profit_before_cost = invest_amount * (np.exp(cumsum_log_return)-1)
            cost = invest_amount * 2 * trading_fee
            profit = profit_before_cost - cost
            trade_symbols.append(symbol)
            trade_types.append("long")
            trade_costs.append(cost)
            trade_profits.append(profit)
            balance += profit #type: ignore
            balances.append(balance)
        
        for symbol in short_symbols:
            cumsum_log_return = merged_df.loc[open_time,:][f"{symbol}_future_return"]
            profit_before_cost = invest_amount * -(np.exp(cumsum_log_return)-1)
            cost = invest_amount * 2 * trading_fee
            profit = profit_before_cost - cost
            trade_symbols.append(symbol)
            trade_types.append("short")
            trade_profits.append(profit)
            trade_costs.append(cost)
            balance += profit #type: ignore
            balances.append(balance)

        times = [open_time for i in range(trade_num)]

        row =pd.DataFrame({"open": times, "symbol":trade_symbols,\
            "type":trade_types,"profit":trade_profits, "cost":trade_costs,"balance":balances})
        
        trade_df = pd.concat([trade_df,row],ignore_index=True)

    trade_df["cumsum_long_profit"] = trade_df[trade_df["type"] == "long"]["profit"].cumsum()
    trade_df["cumsum_short_profit"] = trade_df[trade_df["type"] == "short"]["profit"].cumsum()
    trade_df["cumsum_trading_cost"] = trade_df["cost"].cumsum()
    
    return trade_df

def plot_results(trade_df):
    plt.figure(figsize=(16,9),dpi=720)
    plt.plot(trade_df["open"],trade_df["balance"],c="b",linewidth=1,label="balance")
    plt.plot(trade_df["open"],trade_df["cumsum_long_profit"],c="g",linewidth=1,label="cumsum_long_profit")
    plt.plot(trade_df["open"],trade_df["cumsum_short_profit"],c="r",linewidth=1,label="cumsum_short_profit")
    plt.plot(trade_df["open"],trade_df["cumsum_trading_cost"],c="y",linewidth=1,label="cumsum_trading_cost")
    # plt.plot(trade_df["open"],dfs["BTCUSDT"]["Close"],color="tab:gray",linewidth=1,label="BITCOIN")
    plt.legend()
    plt.show()

#### Improved version
1. Allocation of long and short will be set based on market situation, do more long if bull and vise verse.
2. The holding period will be set dynamicallaly based on market situation. If more volatile, hold a bit shorter???

In [304]:
trade_df = run_strategy(
    dfs = dfs,
    length_df=length_df,
    trade_duration = 600 * 6,
    lookback_period = 14,
    holding_period = 3,
    alpha=0.92,
    avoid_top = 0,
    n_pairs = 5,
    rsi_min = 30,
    rsi_max = 70,
    rsi_period = 14,
    macd_fast = 25,
    macd_slow = 12,
    macd_signal = 9,
    balance=100_000,
    trading_fee=0.0004
)
plot_results(trade_df)

2021-05-18 08:00:00
LONG ['QTUMUSDT', 'BALUSDT', 'BANDUSDT', 'FILUSDT', 'SANDUSDT']
SHORT ['NEARUSDT', 'EOSUSDT', 'XLMUSDT', 'DOGEUSDT', 'ALGOUSDT']
2021-05-18 20:00:00
LONG ['BTCUSDT', 'BALUSDT', 'OGNUSDT', 'LITUSDT', 'UNFIUSDT']
SHORT ['DOTUSDT', 'CHZUSDT', 'ATOMUSDT', 'NEARUSDT', 'GRTUSDT']
2021-05-19 08:00:00
LONG ['FTMUSDT', 'ANKRUSDT', 'SUSHIUSDT', 'MATICUSDT', 'CELRUSDT']
SHORT ['AAVEUSDT', 'DOTUSDT', 'MTLUSDT', 'SNXUSDT', 'HNTUSDT']
2021-05-19 20:00:00
LONG []
SHORT ['CELRUSDT', 'HNTUSDT', 'MATICUSDT', 'ANKRUSDT', 'MKRUSDT']
2021-05-20 08:00:00
LONG ['BTCUSDT', 'WAVESUSDT']
SHORT ['MATICUSDT', 'FILUSDT', 'ADAUSDT', 'CELRUSDT', 'SOLUSDT']
2021-05-20 20:00:00
LONG ['ZECUSDT', 'BALUSDT', 'DASHUSDT', 'LRCUSDT', 'EGLDUSDT']
SHORT ['ANKRUSDT', 'MTLUSDT', 'SOLUSDT', 'HNTUSDT', 'TRXUSDT']
2021-05-21 08:00:00
LONG ['1000SHIBUSDT', 'LITUSDT', 'LINKUSDT', '1INCHUSDT', 'DENTUSDT']
SHORT ['SNXUSDT', 'KSMUSDT', 'SOLUSDT', 'XRPUSDT', 'COTIUSDT']
2021-05-21 20:00:00
LONG ['DASHUSDT', 'LINAUSDT

In [ ]:
dfs["BTCUSDT"].tail(600*6)["Close"].plot()

In [306]:
trade_df.to_csv("../out/out.csv")